In [1]:
import sys
import yaml
with open('..//secrets.yml', 'r') as file:
    secrets = yaml.safe_load(file)
sys.path.append(secrets['elo_proj_path'])

In [2]:
from player_club_classes import team_elo, Player, Club, Match
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly
import pickle

from team_colors import team_color_dict
team_colors = pd.DataFrame(team_color_dict).T
team_colors.columns = ['Primary', 'Secondary']
team_colors = team_colors.rename_axis('Team').reset_index()

from plotly.validators.scatter.marker import SymbolValidator
raw_symbols = SymbolValidator().values

simple_symbols = [i for i in raw_symbols if str(i).isalpha()]

with open('../../Rugby_ELO/processed_data/playerbase.pickle', 'rb') as handle:
    playerbase = pickle.load(handle)
with open('../../Rugby_ELO/processed_data/matchlist.pickle', 'rb') as handle:
    matchlist = pickle.load(handle)
with open('../../Rugby_ELO/processed_data/teamlist.pickle', 'rb') as handle:
    teamlist = pickle.load(handle)

In [3]:
player_elo_list = []
for player_name, player in playerbase.items():
    player_elo = pd.DataFrame(player.elo_list, columns = [
        'Number', 'Full_Name', 'Team', 'Player', 'Position', 'Tries',
        'Try Assists', 'Conversion Goals', 'Penalty Goals',
        'Drop Goals Converted', 'Points', 'Passes', 'Runs', 'Meters Run',
        'Clean Breaks', 'Defenders Beaten', 'Offload', 'Turnovers Conceded',
        'Tackles', 'Missed Tackles', 'Lineouts Won', 'Penalties Conceded',
        'Yellow Cards', 'Red Cards', 'espn_id_num', 'Competition', 'Date',
        'Home Team', 'Home Score', 'Away Team', 'Away Score', 'Minutes',
        'Position_Number', 'gameid', 'Unicode_ID', 'start_elo', 'end_elo'
       ])
    player_elo['Full Name'] = player_name[0]
    player_elo['Unicode_ID'] = player_name[1]
    player_elo_list.append(player_elo)

player_elo = pd.concat(player_elo_list).reset_index(drop=True)
player_elo = pd.merge(player_elo, team_colors, on = 'Team', how = 'left')
player_elo['elo_change'] = player_elo.end_elo - player_elo.start_elo
player_elo.Date = pd.to_datetime(player_elo.Date)

player_elo['week_num'] = player_elo['Date'].dt.isocalendar().week
player_elo['month'] = player_elo['Date'].dt.month
player_elo['year'] = player_elo['Date'].dt.year

In [103]:
def percentile(group):
    sz = group.size-1
    ranks = group.rank(method='max')
    return np.floor(100.0*(ranks-1)/sz)

starters = player_elo[player_elo.Position != 'R']
starters = starters.dropna(subset=['Position'])
current_players = starters[starters.groupby(['Full Name'])['Date'].transform(max) == starters['Date']]
current_players['percentile'] = np.floor(current_players.groupby('Position')['end_elo'].apply(percentile))
current_players = current_players[['Full_Name', 'Unicode_ID', 'percentile']]
current_players

C:\Users\Graeham\AppData\Local\Temp\ipykernel_9368\1572876672.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_players['percentile'] = np.floor(current_players.groupby('Position')['end_elo'].apply(percentile))


,Full_Name,Unicode_ID,percentile
264,Nick Abendanon,55485151,97.0
401,Mike Tindall,49485155,81.0
550,Anthony Allen,55545151,82.0
568,James Bailey,53485349,38.0
800,Ryan Lamb,55515350,48.0
...,...,...,...
503971,Nahuel Milan,1101119510111511211095105100327897104117101108...,6.0
503972,Geronimo Prisciantelli,1101119510111511211095105100327110111411111010...,6.0
503976,Konstantine Mikautadze,1101119510111511211095105100327511111011511697...,11.0
503977,Aka Tabutsadze,1101119510111511211095105100326510797328497981...,13.0


In [79]:
match_list = []
for _, match in matchlist.items():
    match_list.append({key:val for key, val in vars(match).items()})

In [80]:
future_games = [x for x in match_list if 'point_diff' not in x.keys()]

In [82]:
future_game

{'home_team_name': 'Japan',
 'away_team_name': 'France',
 'home_team': array([[15.0, 'Ryohei Yamanaka', 'Japan', 'R Yamanaka', 'FB', 0.0, 0.0,
         0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
         0.0, 0.0, 0.0, 0.0, '22646', 'International Test Match 2022',
         Timestamp('2022-07-08 14:50:00'), 'Japan', nan, 'France', nan,
         nan, nan, '2:50 PM, July 8, 2022 @ Japan', '5050545254',
         105.64167761167195],
        [23.0, 'Shane Gates', 'Japan', 'S Gates', 'R', 0.0, 0.0, 0.0, 0.0,
         0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
         0.0, 0.0, '18971', 'International Test Match 2022',
         Timestamp('2022-07-08 14:50:00'), 'Japan', nan, 'France', nan,
         nan, nan, '2:50 PM, July 8, 2022 @ Japan', '4956575549',
         83.19163501578386],
        [14.0, 'Gerhard van den Heever', 'Japan', 'G Heever', 'W', 0.0,
         0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
         0.0, 0.0, 0.

In [104]:
future_game = future_games[0]

home_team = pd.DataFrame(future_game['home_team'][:, [0,1,-2, -1]], columns = ['Number', 'Full_Name', 'Unicode_ID', 'elo'])
home_team = home_team.merge(current_players, on=['Full_Name', 'Unicode_ID'])
home_team = home_team.drop(['Unicode_ID'], axis = 1)
away_team = pd.DataFrame(future_game['away_team'][:, [0,1,-2, -1]], columns = ['Number', 'Full_Name', 'Unicode_ID', 'elo'])
away_team = away_team.merge(current_players, on=['Full_Name', 'Unicode_ID'])
away_team = away_team.drop(['Unicode_ID'], axis = 1)

home_team.columns = ['Number', 'Home Name', 'Home elo', 'Home Percentile']
away_team.columns = ['Number', 'Away Name', 'Away elo', 'Away Percentile']

all_players = pd.merge(home_team, away_team)
all_players = all_players.sort_values('Number')
all_players


,Number,Home Name,Home elo,Home Percentile,Away Name,Away elo,Away Percentile
7,1.0,Keita Inagaki,95.693948,34.0,Jean-Baptiste Gros,114.835326,96.0
8,2.0,Atsushi Sakate,103.016034,79.0,Peato Mauvaka,108.293691,92.0
9,3.0,Asaeli Ai Valu,109.901054,93.0,Demba Bamba,106.963526,88.0
10,4.0,Warner Dearns,81.632401,13.0,Thibaud Flament,91.362326,25.0
13,5.0,Sanaila Waqa,98.572695,56.0,Thomas Jolmes,93.856298,32.0
12,6.0,Michael Leitch,111.575428,94.0,Dylan Cretin,89.515212,24.0
20,7.0,Ben Gunter,74.382303,1.0,Charles Ollivon,124.006521,99.0
19,8.0,Jack Cornelsen,100.25964,68.0,Yoan Tanga,103.366735,76.0
11,9.0,Naoto Saito,77.317966,2.0,Maxime Lucu,88.440914,19.0
6,10.0,Seungsin Lee,78.591693,6.0,Matthieu Jalibert,110.788907,92.0


In [95]:
current_players.head()

,Full_Name,Unicode_ID,percentile
264,Nick Abendanon,55485151,97.0
401,Mike Tindall,49485155,81.0
550,Anthony Allen,55545151,82.0
568,James Bailey,53485349,38.0
800,Ryan Lamb,55515350,48.0


In [6]:
month = match_df[(match_df.year == 2022) & (match_df.month == 7)]
closest_games = month.sort_values('lineup_spread',  key=abs).head()

sorted_pred_accuracy = month.assign(tmp=month["point_diff"] - month["lineup_spread"]).sort_values(by="tmp", key=abs)#.drop(columns="tmp")
best_preds = sorted_pred_accuracy.head()
worst_preds = sorted_pred_accuracy.tail()

biggest_upsets = month[(month.lineup_spread * month.point_diff) < 0].assign(tmp=month["point_diff"] - month["lineup_spread"]).sort_values(by="tmp", key=abs, ascending=False).drop(columns="tmp").head()
biggest_covers = month[((month.lineup_spread * month.point_diff) > 0)].assign(tmp=month["point_diff"] - month["lineup_spread"]).sort_values(by="tmp", key=abs, ascending=False)#.drop(columns="tmp").head()

In [7]:
# CHANGE
player_month = player_elo[(player_elo.year == 2022) & (player_elo.month == 5)]
oneplayer = player_month[(player_month.Unicode_ID == '5155544848') & (player_month['Full Name'] == 'Caleb Delany')].sort_values('Date')
oneplayer.iloc[-1].end_elo - oneplayer.iloc[0].start_elo

-2.554117441069806

In [11]:
def elo_change(df):
    return df.iloc[-1].end_elo, df.iloc[0].start_elo, df.iloc[-1].end_elo - df.iloc[0].start_elo

elo_changes = player_month.sort_values('Date').groupby(['Unicode_ID', 'Full Name', 'Team']).apply(elo_change).reset_index()
elo_changes[['end_elo', 'start_elo', 'elo_change']] = pd.DataFrame(elo_changes[0].tolist(), index=elo_changes.index)
elo_changes.sort_values('elo_change')

,Unicode_ID,Full Name,Team,0,end_elo,start_elo,elo_change
576,5051515156,James Blackwell,Hurricanes,"(100.88391371265058, 106.42990302317858, -5.54...",100.883914,106.429903,-5.545989
826,5055485550,Salesi Rayasi,Hurricanes,"(110.70188334521939, 115.67822631062886, -4.97...",110.701883,115.678226,-4.976343
1070,5149525654,Quinn Tupaea,Chiefs,"(113.26707089934938, 118.08496318931884, -4.81...",113.267071,118.084963,-4.817892
267,5048485452,Ardie Savea,Hurricanes,"(118.33886733853684, 122.95562106235826, -4.61...",118.338867,122.955621,-4.616754
589,5051515748,Josh Ioane,Chiefs,"(98.22976447566921, 102.77928260508381, -4.549...",98.229764,102.779283,-4.549518
...,...,...,...,...,...,...,...
406,5049535052,Mattia Bellini,Benetton Treviso,"(96.79422627952788, 92.30762615721996, 4.48660...",96.794226,92.307626,4.486600
745,5053505254,Rhyno Smith,Benetton Treviso,"(99.09602275738854, 94.60942263508062, 4.48660...",99.096023,94.609423,4.486600
244,4957555455,Juan Ignacio Brex,Benetton Treviso,"(98.82188475913395, 94.33528463682603, 4.48660...",98.821885,94.335285,4.486600
408,5049535155,Federico Ruzza,Benetton Treviso,"(102.4315904715746, 97.94499034926667, 4.48660...",102.431590,97.944990,4.486600
